In [1]:
import pandas as pd

crime_data = pd.read_csv('crime_ddn_city.csv', encoding='utf-8')
crime_data.head()

,lng,lat,date,customdate,time,customtime,crime,barangay,city,region,province,modus,crimetype,station,moduscode,date_com,datechar
0,125.808,7.44808,30/07/2017,"Sun - Jul 30, 2017",16:11:00,4:11:PM,Vehicular Traffic Accident,Magugpo Poblacion,Tagum City,110000000,112300000,Not Indicated,123,Tagum CPS,mc00,30/07/2017,42946
1,125.803,7.44675,30/07/2017,"Sun - Jul 30, 2017",0:20:00,12:20:AM,MURDER,Magugpo Poblacion,Tagum City,110000000,112300000,Stabbing/Hacking,2,Tagum CPS,mc90,30/07/2017,42946
2,125.803,7.44897,30/07/2017,"Sun - Jul 30, 2017",0:20:00,12:20:AM,PHYSICAL INJURIES,Magugpo Poblacion,Tagum City,110000000,112300000,Stabbing/Hacking,6,Tagum CPS,mc94,30/07/2017,42946
3,125.818,7.45982,30/07/2017,"Sun - Jul 30, 2017",20:50:00,8:50:PM,MURDER,Magugpo North,Tagum City,110000000,112300000,Shooting,2,Tagum CPS,mc89,30/07/2017,42946
4,125.818,7.46016,30/07/2017,"Sun - Jul 30, 2017",20:50:00,8:50:PM,PHYSICAL INJURIES,Magugpo North,Tagum City,110000000,112300000,Shooting,6,Tagum CPS,mc93,30/07/2017,42946


# Crime, Weather, and Daylight

In [2]:
weather_data = pd.read_csv('weather_davaodelnorte_data.csv', encoding='utf-8')
weather_data.head()

,Region,Date,Time,Weather,Temp (deg celsius),Feels Like (deg celsius),Wind (km/h),Direction,Cloud %,Humidity %,Precip (mm),Pressure (mb)
0,Panabo City,01/08/2017,0:00,Partly cloudy,29,29,1,SSE,41%,79%,0.3,"1,011"
1,Panabo City,01/08/2017,3:00,Partly cloudy,30,27,2,NNW,48%,80%,0.0,"1,011"
2,Panabo City,01/08/2017,6:00,Partly cloudy,32,29,1,W,42%,77%,0.0,"1,011"
3,Panabo City,01/08/2017,9:00,Partly cloudy,34,37,4,E,49%,68%,0.0,"1,011"
4,Panabo City,01/08/2017,12:00,Light rain shower,35,44,10,SSE,62%,58%,0.1,"1,010"


In [3]:
day_night_data = pd.read_csv('sunrise_sunset_davaodelnorte_data.csv', encoding='utf-8')
day_night_data.head()

,Region,Date,Sunrise,Sunset
0,Davao del norte,30/07/2017,5:31 AM,5:57 PM
1,Davao del norte,31/07/2017,5:31 AM,5:57 PM
2,Davao del norte,01/08/2017,5:31 AM,5:57 PM
3,Davao del norte,02/08/2017,5:31 AM,5:57 PM
4,Davao del norte,03/08/2017,5:31 AM,5:56 PM


In [4]:
weather_sun_data = pd.merge(weather_data, day_night_data, how='left', left_on=['Date'], right_on=['Date'])
weather_sun_data = weather_sun_data[['Region_x','Date','Time','Weather','Temp (deg celsius)',
                                     'Feels Like (deg celsius)','Wind (km/h)','Direction','Cloud %',
                                     'Humidity %','Precip (mm)','Pressure (mb)','Sunrise','Sunset']]
weather_sun_data.head()

,Region_x,Date,Time,Weather,Temp (deg celsius),Feels Like (deg celsius),Wind (km/h),Direction,Cloud %,Humidity %,Precip (mm),Pressure (mb),Sunrise,Sunset
0,Panabo City,01/08/2017,0:00,Partly cloudy,29,29,1,SSE,41%,79%,0.3,"1,011",5:31 AM,5:57 PM
1,Panabo City,01/08/2017,3:00,Partly cloudy,30,27,2,NNW,48%,80%,0.0,"1,011",5:31 AM,5:57 PM
2,Panabo City,01/08/2017,6:00,Partly cloudy,32,29,1,W,42%,77%,0.0,"1,011",5:31 AM,5:57 PM
3,Panabo City,01/08/2017,9:00,Partly cloudy,34,37,4,E,49%,68%,0.0,"1,011",5:31 AM,5:57 PM
4,Panabo City,01/08/2017,12:00,Light rain shower,35,44,10,SSE,62%,58%,0.1,"1,010",5:31 AM,5:57 PM


## Data Merging

In [5]:
import datetime
def convert_to_weather_time(crime_time):
    time_str_format = "%H:%M:%S"
    time_weather_str_format = "%H:%M"
    time = datetime.datetime.strptime(str(crime_time), time_str_format)
    weather_times = ["0:00","3:00","6:00","9:00",
                     "12:00","15:00","18:00","21:00"]
    x = 0
    for each in weather_times:
        if x == 0:
            x+=1
            continue
        weather_time = datetime.datetime.strptime(each, time_weather_str_format)
        if weather_time > time:
            return weather_times[x-1]
        if x == 7 and weather_time <= time:
            return weather_times[x]
        x+=1
def convert_to_time_bin(crime_time):
    time_str_format = "%H:%M:%S"
    time = datetime.datetime.strptime(str(crime_time), time_str_format).hour
    time_bins = ['00:00 - 00:59','01:00 - 01:59','02:00 - 02:59','03:00 - 03:59',
                 '04:00 - 04:59','05:00 - 05:59','06:00 - 06:59','07:00 - 07:59',
                 '08:00 - 08:59','09:00 - 09:59','10:00 - 10:59','11:00 - 11:59',
                 '12:00 - 12:59','13:00 - 13:59','14:00 - 14:59','15:00 - 15:59',
                 '16:00 - 16:59','17:00 - 17:59','18:00 - 18:59','19:00 - 19:59',
                 '20:00 - 20:59','21:00 - 21:59','22:00 - 22:59','23:00 - 23:59',
                ]
    return time_bins[time]
def convert_to_day(crime_date):
    date_str_format = "%d/%m/%Y"
    day = datetime.datetime.strptime(str(crime_date), date_str_format)
    return day.strftime("%A")

In [6]:
crime_data_temp = crime_data.copy()
crime_data_temp['weathertime'] = crime_data.apply(lambda row: convert_to_weather_time(row['time']),axis=1)
crime_data_temp['time_bin'] = crime_data.apply(lambda row: convert_to_time_bin(row['time']),axis=1)
crime_data_temp['day'] = crime_data.apply(lambda row: convert_to_day(row['date']),axis=1)
crime_data_temp.head()

,lng,lat,date,customdate,time,customtime,crime,barangay,city,region,province,modus,crimetype,station,moduscode,date_com,datechar,weathertime,time_bin,day
0,125.808,7.44808,30/07/2017,"Sun - Jul 30, 2017",16:11:00,4:11:PM,Vehicular Traffic Accident,Magugpo Poblacion,Tagum City,110000000,112300000,Not Indicated,123,Tagum CPS,mc00,30/07/2017,42946,15:00,16:00 - 16:59,Sunday
1,125.803,7.44675,30/07/2017,"Sun - Jul 30, 2017",0:20:00,12:20:AM,MURDER,Magugpo Poblacion,Tagum City,110000000,112300000,Stabbing/Hacking,2,Tagum CPS,mc90,30/07/2017,42946,0:00,00:00 - 00:59,Sunday
2,125.803,7.44897,30/07/2017,"Sun - Jul 30, 2017",0:20:00,12:20:AM,PHYSICAL INJURIES,Magugpo Poblacion,Tagum City,110000000,112300000,Stabbing/Hacking,6,Tagum CPS,mc94,30/07/2017,42946,0:00,00:00 - 00:59,Sunday
3,125.818,7.45982,30/07/2017,"Sun - Jul 30, 2017",20:50:00,8:50:PM,MURDER,Magugpo North,Tagum City,110000000,112300000,Shooting,2,Tagum CPS,mc89,30/07/2017,42946,18:00,20:00 - 20:59,Sunday
4,125.818,7.46016,30/07/2017,"Sun - Jul 30, 2017",20:50:00,8:50:PM,PHYSICAL INJURIES,Magugpo North,Tagum City,110000000,112300000,Shooting,6,Tagum CPS,mc93,30/07/2017,42946,18:00,20:00 - 20:59,Sunday


In [7]:
crime_weather = pd.merge(crime_data_temp, weather_sun_data, how='left', left_on=['city','date','weathertime'], right_on=['Region_x','Date','Time'])
crime_weather.drop(crime_weather.columns[[5,9,10,12,13,14,15,16]],axis=1,inplace=True)
crime_weather.head()
# crime_weather.describe()

,lng,lat,date,customdate,time,crime,barangay,city,modus,weathertime,...,Temp (deg celsius),Feels Like (deg celsius),Wind (km/h),Direction,Cloud %,Humidity %,Precip (mm),Pressure (mb),Sunrise,Sunset
0,125.808,7.44808,30/07/2017,"Sun - Jul 30, 2017",16:11:00,Vehicular Traffic Accident,Magugpo Poblacion,Tagum City,Not Indicated,15:00,...,30,32,15,NNE,57%,58%,0.1,"1,012",5:31 AM,5:57 PM
1,125.803,7.44675,30/07/2017,"Sun - Jul 30, 2017",0:20:00,MURDER,Magugpo Poblacion,Tagum City,Stabbing/Hacking,0:00,...,23,25,8,ESE,18%,89%,0.0,"1,013",5:31 AM,5:57 PM
2,125.803,7.44897,30/07/2017,"Sun - Jul 30, 2017",0:20:00,PHYSICAL INJURIES,Magugpo Poblacion,Tagum City,Stabbing/Hacking,0:00,...,23,25,8,ESE,18%,89%,0.0,"1,013",5:31 AM,5:57 PM
3,125.818,7.45982,30/07/2017,"Sun - Jul 30, 2017",20:50:00,MURDER,Magugpo North,Tagum City,Shooting,18:00,...,27,29,13,NNE,1%,66%,0.0,"1,013",5:31 AM,5:57 PM
4,125.818,7.46016,30/07/2017,"Sun - Jul 30, 2017",20:50:00,PHYSICAL INJURIES,Magugpo North,Tagum City,Shooting,18:00,...,27,29,13,NNE,1%,66%,0.0,"1,013",5:31 AM,5:57 PM


In [8]:
import datetime
def set_day_night(crime_time,sunrise,sunset):
    time_str_format = "%H:%M:%S"
    time_sunrise_sunset_str_format = "%I:%M %p"
    time = datetime.datetime.strptime(str(crime_time), time_str_format)
    rise = datetime.datetime.strptime(str(sunrise), time_sunrise_sunset_str_format)
    snset = datetime.datetime.strptime(str(sunset), time_sunrise_sunset_str_format)
    
    if time < rise or time > snset:
        return "night"
    else:
        return "day"

In [9]:
crime_weather['day or night'] = crime_weather.apply(lambda row: set_day_night(row['time'],row['Sunrise'],row['Sunset']),axis=1)
crime_weather.to_csv('generated/crime_weather.csv')

# Crime and Places

In [10]:
places_data = pd.read_csv('vertical_places_crime.csv', encoding='utf-8')
places_data.head()

,lat,lng,1_nearest_feature,1_nearest_feature_distance,2_nearest_feature,2_nearest_feature_distance,3_nearest_feature,3_nearest_feature_distance,4_nearest_feature,4_nearest_feature_distance,...,Reservior,Residential Building,Resort,Restaurant,River,Road,School,Sports Center,Tourist Attraction,Transport Terminal
0,7.44808,125.808,Highway Primary,10,Highway Tertiary,10,Clinic,30,Commercial Building,40,...,-,-,-,150,-,-,-,-,-,-
1,7.44675,125.803,Highway Tertiary,10,Private Office,20,Church,70,Park,70,...,-,-,-,70,-,-,100,-,-,-
2,7.44897,125.803,Highway Tertiary,30,Highway Pedestrian,40,Greenfield,60,School,60,...,-,-,-,130,-,-,60,-,90,-
3,7.45982,125.818,Highway Residential,40,Highway Tertiary,40,Gas Station,90,Highway Pedestrian,120,...,-,-,-,160,-,-,180,-,-,-
4,7.46016,125.818,Highway Residential,20,Highway Tertiary,60,Gas Station,130,Highway Pedestrian,140,...,-,-,-,160,-,-,160,-,-,-


## Nearest places to the crime scene

In [11]:
places_nearest = places_data.copy()
places_nearest = places_nearest.iloc[:,:12]
places_nearest.head()

,lat,lng,1_nearest_feature,1_nearest_feature_distance,2_nearest_feature,2_nearest_feature_distance,3_nearest_feature,3_nearest_feature_distance,4_nearest_feature,4_nearest_feature_distance,5_nearest_feature,5_nearest_feature_distance
0,7.44808,125.808,Highway Primary,10,Highway Tertiary,10,Clinic,30,Commercial Building,40,Gas Station,50
1,7.44675,125.803,Highway Tertiary,10,Private Office,20,Church,70,Park,70,Restaurant,70
2,7.44897,125.803,Highway Tertiary,30,Highway Pedestrian,40,Greenfield,60,School,60,Commercial Building,70
3,7.45982,125.818,Highway Residential,40,Highway Tertiary,40,Gas Station,90,Highway Pedestrian,120,Highway,130
4,7.46016,125.818,Highway Residential,20,Highway Tertiary,60,Gas Station,130,Highway Pedestrian,140,Highway,150


In [12]:
crime_data_temp = crime_data.copy()
crime_data_temp['time_bin'] = crime_data.apply(lambda row: convert_to_time_bin(row['time']),axis=1)
crime_data_temp['day'] = crime_data.apply(lambda row: convert_to_day(row['date']),axis=1)
crime_data_temp.head()

,lng,lat,date,customdate,time,customtime,crime,barangay,city,region,province,modus,crimetype,station,moduscode,date_com,datechar,time_bin,day
0,125.808,7.44808,30/07/2017,"Sun - Jul 30, 2017",16:11:00,4:11:PM,Vehicular Traffic Accident,Magugpo Poblacion,Tagum City,110000000,112300000,Not Indicated,123,Tagum CPS,mc00,30/07/2017,42946,16:00 - 16:59,Sunday
1,125.803,7.44675,30/07/2017,"Sun - Jul 30, 2017",0:20:00,12:20:AM,MURDER,Magugpo Poblacion,Tagum City,110000000,112300000,Stabbing/Hacking,2,Tagum CPS,mc90,30/07/2017,42946,00:00 - 00:59,Sunday
2,125.803,7.44897,30/07/2017,"Sun - Jul 30, 2017",0:20:00,12:20:AM,PHYSICAL INJURIES,Magugpo Poblacion,Tagum City,110000000,112300000,Stabbing/Hacking,6,Tagum CPS,mc94,30/07/2017,42946,00:00 - 00:59,Sunday
3,125.818,7.45982,30/07/2017,"Sun - Jul 30, 2017",20:50:00,8:50:PM,MURDER,Magugpo North,Tagum City,110000000,112300000,Shooting,2,Tagum CPS,mc89,30/07/2017,42946,20:00 - 20:59,Sunday
4,125.818,7.46016,30/07/2017,"Sun - Jul 30, 2017",20:50:00,8:50:PM,PHYSICAL INJURIES,Magugpo North,Tagum City,110000000,112300000,Shooting,6,Tagum CPS,mc93,30/07/2017,42946,20:00 - 20:59,Sunday


In [13]:
places_nearest = pd.merge(crime_data_temp, places_nearest, how='left', left_on=['lat','lng'], right_on=['lat','lng'])
places_nearest.drop(places_nearest.columns[[5,9,10,12,13,14,15,16]],axis=1,inplace=True)
for index, row in places_nearest.iterrows():
    for col in places_nearest:
        if row[col] == '-':
            places_nearest.at[index,col] = None
places_nearest.to_csv('generated/crime_places_nearest.csv')
places_nearest.head()

,lng,lat,date,customdate,time,crime,barangay,city,modus,time_bin,...,1_nearest_feature,1_nearest_feature_distance,2_nearest_feature,2_nearest_feature_distance,3_nearest_feature,3_nearest_feature_distance,4_nearest_feature,4_nearest_feature_distance,5_nearest_feature,5_nearest_feature_distance
0,125.808,7.44808,30/07/2017,"Sun - Jul 30, 2017",16:11:00,Vehicular Traffic Accident,Magugpo Poblacion,Tagum City,Not Indicated,16:00 - 16:59,...,Highway Primary,10,Highway Tertiary,10,Clinic,30,Commercial Building,40,Gas Station,50
1,125.803,7.44675,30/07/2017,"Sun - Jul 30, 2017",0:20:00,MURDER,Magugpo Poblacion,Tagum City,Stabbing/Hacking,00:00 - 00:59,...,Highway Tertiary,10,Private Office,20,Church,70,Park,70,Restaurant,70
2,125.803,7.44897,30/07/2017,"Sun - Jul 30, 2017",0:20:00,PHYSICAL INJURIES,Magugpo Poblacion,Tagum City,Stabbing/Hacking,00:00 - 00:59,...,Highway Tertiary,30,Highway Pedestrian,40,Greenfield,60,School,60,Commercial Building,70
3,125.818,7.45982,30/07/2017,"Sun - Jul 30, 2017",20:50:00,MURDER,Magugpo North,Tagum City,Shooting,20:00 - 20:59,...,Highway Residential,40,Highway Tertiary,40,Gas Station,90,Highway Pedestrian,120,Highway,130
4,125.818,7.46016,30/07/2017,"Sun - Jul 30, 2017",20:50:00,PHYSICAL INJURIES,Magugpo North,Tagum City,Shooting,20:00 - 20:59,...,Highway Residential,20,Highway Tertiary,60,Gas Station,130,Highway Pedestrian,140,Highway,150


## Place presence

In [14]:
places_presence = places_data.copy()
places_presence.drop(places_presence.columns[[2,3,4,5,6,7,8,9,10,11]],axis=1,inplace=True)
places_presence.head()

,lat,lng,Bank,Bar,Beach,Bridge,Canal,Cemetery,Church,Clinic,...,Reservior,Residential Building,Resort,Restaurant,River,Road,School,Sports Center,Tourist Attraction,Transport Terminal
0,7.44808,125.808,160,-,-,-,-,-,70,30,...,-,-,-,150,-,-,-,-,-,-
1,7.44675,125.803,120,130,-,-,-,-,70,-,...,-,-,-,70,-,-,100,-,-,-
2,7.44897,125.803,120,-,-,-,-,-,-,-,...,-,-,-,130,-,-,60,-,90,-
3,7.45982,125.818,-,-,-,-,-,-,170,-,...,-,-,-,160,-,-,180,-,-,-
4,7.46016,125.818,-,-,-,-,-,-,190,-,...,-,-,-,160,-,-,160,-,-,-


In [15]:
places_presence = pd.merge(crime_data_temp, places_presence, how='left', left_on=['lat','lng'], right_on=['lat','lng'])
places_presence.drop(places_presence.columns[[5,9,10,12,13,14,15,16]],axis=1,inplace=True)
for index, row in places_presence.iterrows():
    for col in places_presence:
        if row[col] == '-':
            places_presence.at[index,col] = None
places_presence.to_csv('generated/crime_places_presence.csv')
places_presence.head()

,lng,lat,date,customdate,time,crime,barangay,city,modus,time_bin,...,Reservior,Residential Building,Resort,Restaurant,River,Road,School,Sports Center,Tourist Attraction,Transport Terminal
0,125.808,7.44808,30/07/2017,"Sun - Jul 30, 2017",16:11:00,Vehicular Traffic Accident,Magugpo Poblacion,Tagum City,Not Indicated,16:00 - 16:59,...,None,None,None,150,None,None,None,None,None,None
1,125.803,7.44675,30/07/2017,"Sun - Jul 30, 2017",0:20:00,MURDER,Magugpo Poblacion,Tagum City,Stabbing/Hacking,00:00 - 00:59,...,None,None,None,70,None,None,100,None,None,None
2,125.803,7.44897,30/07/2017,"Sun - Jul 30, 2017",0:20:00,PHYSICAL INJURIES,Magugpo Poblacion,Tagum City,Stabbing/Hacking,00:00 - 00:59,...,None,None,None,130,None,None,60,None,90,None
3,125.818,7.45982,30/07/2017,"Sun - Jul 30, 2017",20:50:00,MURDER,Magugpo North,Tagum City,Shooting,20:00 - 20:59,...,None,None,None,160,None,None,180,None,None,None
4,125.818,7.46016,30/07/2017,"Sun - Jul 30, 2017",20:50:00,PHYSICAL INJURIES,Magugpo North,Tagum City,Shooting,20:00 - 20:59,...,None,None,None,160,None,None,160,None,None,None


### Places within 50, 100, 150, 200, 300, 400, 500 meters to crime spot

In [16]:
places_presence_50 = places_presence.copy()
for index, row in places_presence_50.iterrows():
    for col in places_presence_50:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if row[col] == None or int(row[col]) > 50:
                places_presence_50.at[index,col] = 0
            else:
                places_presence_50.at[index,col] = 1
drop_rows = []
for index, row in places_presence_50.iterrows():
    dropthis_row = True
    for col in places_presence_50:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if int(row[col]) == 1:
                dropthis_row = False
    if dropthis_row == True:
        drop_rows.append(index)
places_presence_50.drop(places_presence_50.index[drop_rows], inplace=True)
places_presence_50.to_csv('generated/crime_places_presence_50.csv')

ValueError: invalid literal for int() with base 10: '16:00 - 16:59'

In [ ]:
places_presence_100 = places_presence.copy()
for index, row in places_presence_100.iterrows():
    for col in places_presence_100:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if row[col] == None or int(row[col]) > 100:
                places_presence_100.at[index,col] = 0
            else:
                places_presence_100.at[index,col] = 1
drop_rows = []
for index, row in places_presence_100.iterrows():
    dropthis_row = True
    for col in places_presence_100:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if int(row[col]) == 1:
                dropthis_row = False
    if dropthis_row == True:
        drop_rows.append(index)
places_presence_100.drop(places_presence_100.index[drop_rows], inplace=True)
places_presence_100.to_csv('generated/crime_places_presence_100.csv')

In [ ]:
places_presence_150 = places_presence.copy()
for index, row in places_presence_150.iterrows():
    for col in places_presence_150:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if row[col] == None or int(row[col]) > 150:
                places_presence_150.at[index,col] = 0
            else:
                places_presence_150.at[index,col] = 1
drop_rows = []
for index, row in places_presence_150.iterrows():
    dropthis_row = True
    for col in places_presence_150:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if int(row[col]) == 1:
                dropthis_row = False
    if dropthis_row == True:
        drop_rows.append(index)
places_presence_150.drop(places_presence_150.index[drop_rows], inplace=True)
places_presence_150.to_csv('generated/crime_places_presence_150.csv')

In [ ]:
places_presence_200 = places_presence.copy()
for index, row in places_presence_200.iterrows():
    for col in places_presence_200:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if row[col] == None or int(row[col]) > 200:
                places_presence_200.at[index,col] = 0
            else:
                places_presence_200.at[index,col] = 1
drop_rows = []
for index, row in places_presence_200.iterrows():
    dropthis_row = True
    for col in places_presence_200:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if int(row[col]) == 1:
                dropthis_row = False
    if dropthis_row == True:
        drop_rows.append(index)
places_presence_200.drop(places_presence_200.index[drop_rows], inplace=True)
places_presence_200.to_csv('generated/crime_places_presence_200.csv')

In [ ]:
places_presence_300 = places_presence.copy()
for index, row in places_presence_300.iterrows():
    for col in places_presence_300:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if row[col] == None or int(row[col]) > 300:
                places_presence_300.at[index,col] = 0
            else:
                places_presence_300.at[index,col] = 1
drop_rows = []
for index, row in places_presence_300.iterrows():
    dropthis_row = True
    for col in places_presence_300:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if int(row[col]) == 1:
                dropthis_row = False
    if dropthis_row == True:
        drop_rows.append(index)
places_presence_300.drop(places_presence_300.index[drop_rows], inplace=True)
places_presence_300.to_csv('generated/crime_places_presence_300.csv')

In [ ]:
places_presence_400 = places_presence.copy()
for index, row in places_presence_400.iterrows():
    for col in places_presence_400:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if row[col] == None or int(row[col]) > 400:
                places_presence_400.at[index,col] = 0
            else:
                places_presence_400.at[index,col] = 1
drop_rows = []
for index, row in places_presence_400.iterrows():
    dropthis_row = True
    for col in places_presence_400:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if int(row[col]) == 1:
                dropthis_row = False
    if dropthis_row == True:
        drop_rows.append(index)
places_presence_400.drop(places_presence_400.index[drop_rows], inplace=True)
places_presence_400.to_csv('generated/crime_places_presence_400.csv')

In [ ]:
places_presence_500 = places_presence.copy()
for index, row in places_presence_500.iterrows():
    for col in places_presence_500:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if row[col] == None or int(row[col]) > 500:
                places_presence_500.at[index,col] = 0
            else:
                places_presence_500.at[index,col] = 1
for index, row in places_presence_500.iterrows():
    dropthis_row = True
    for col in places_presence_500:
        if col not in ['lng','lat','date','customdate','time','crime','barangay','city','modus']:
            if int(row[col]) == 1:
                dropthis_row = False
    if dropthis_row == True:
        drop_rows.append(index)
places_presence_500.drop(places_presence_500.index[drop_rows], inplace=True)
places_presence_500.to_csv('generated/crime_places_presence_500.csv')

# Crime and SocioEconomic Factors

## Currency and PSE index

In [17]:
phpusd_psei = pd.read_csv('php_to_usd_data.csv', encoding='utf-8')
phpusd_psei.head()

,date,php-usd-rate,PSE-index-Open,PSE-index-High,PSE-index-Low,PSE-index-Close,PSE-index-Adj Close,PSE-index-Volume
0,30/07/2017,50.546,8075.720215,8079.700195,8046.759766,8071.470215,8071.470215,115600
1,31/07/2017,50.582,8068.660156,8070.040039,7953.470215,8018.049805,8018.049805,175300
2,01/08/2017,50.502,8015.680176,8017.180176,7906.600098,7906.600098,7906.600098,162200
3,02/08/2017,50.397,7936.890137,8037.910156,7872.649902,7872.649902,7872.649902,154400
4,03/08/2017,50.422,7897.839844,7946.310059,7862.109863,7876.660156,7876.660156,114100


In [18]:
crime_data_temp = crime_data.copy()
crime_data_temp['time_bin'] = crime_data.apply(lambda row: convert_to_time_bin(row['time']),axis=1)
crime_data_temp['day'] = crime_data.apply(lambda row: convert_to_day(row['date']),axis=1)
crime_data_temp.head()

,lng,lat,date,customdate,time,customtime,crime,barangay,city,region,province,modus,crimetype,station,moduscode,date_com,datechar,time_bin,day
0,125.808,7.44808,30/07/2017,"Sun - Jul 30, 2017",16:11:00,4:11:PM,Vehicular Traffic Accident,Magugpo Poblacion,Tagum City,110000000,112300000,Not Indicated,123,Tagum CPS,mc00,30/07/2017,42946,16:00 - 16:59,Sunday
1,125.803,7.44675,30/07/2017,"Sun - Jul 30, 2017",0:20:00,12:20:AM,MURDER,Magugpo Poblacion,Tagum City,110000000,112300000,Stabbing/Hacking,2,Tagum CPS,mc90,30/07/2017,42946,00:00 - 00:59,Sunday
2,125.803,7.44897,30/07/2017,"Sun - Jul 30, 2017",0:20:00,12:20:AM,PHYSICAL INJURIES,Magugpo Poblacion,Tagum City,110000000,112300000,Stabbing/Hacking,6,Tagum CPS,mc94,30/07/2017,42946,00:00 - 00:59,Sunday
3,125.818,7.45982,30/07/2017,"Sun - Jul 30, 2017",20:50:00,8:50:PM,MURDER,Magugpo North,Tagum City,110000000,112300000,Shooting,2,Tagum CPS,mc89,30/07/2017,42946,20:00 - 20:59,Sunday
4,125.818,7.46016,30/07/2017,"Sun - Jul 30, 2017",20:50:00,8:50:PM,PHYSICAL INJURIES,Magugpo North,Tagum City,110000000,112300000,Shooting,6,Tagum CPS,mc93,30/07/2017,42946,20:00 - 20:59,Sunday


In [19]:
currency_pse = pd.merge(crime_data_temp, phpusd_psei, how='left', left_on=['date'], right_on=['date'])
currency_pse.drop(currency_pse.columns[[5,9,10,12,13,14,15,16]],axis=1,inplace=True)
currency_pse.to_csv('generated/crime_currency_psei.csv')
currency_pse.head()

,lng,lat,date,customdate,time,crime,barangay,city,modus,time_bin,day,php-usd-rate,PSE-index-Open,PSE-index-High,PSE-index-Low,PSE-index-Close,PSE-index-Adj Close,PSE-index-Volume
0,125.808,7.44808,30/07/2017,"Sun - Jul 30, 2017",16:11:00,Vehicular Traffic Accident,Magugpo Poblacion,Tagum City,Not Indicated,16:00 - 16:59,Sunday,50.546,8075.720215,8079.700195,8046.759766,8071.470215,8071.470215,115600
1,125.803,7.44675,30/07/2017,"Sun - Jul 30, 2017",0:20:00,MURDER,Magugpo Poblacion,Tagum City,Stabbing/Hacking,00:00 - 00:59,Sunday,50.546,8075.720215,8079.700195,8046.759766,8071.470215,8071.470215,115600
2,125.803,7.44897,30/07/2017,"Sun - Jul 30, 2017",0:20:00,PHYSICAL INJURIES,Magugpo Poblacion,Tagum City,Stabbing/Hacking,00:00 - 00:59,Sunday,50.546,8075.720215,8079.700195,8046.759766,8071.470215,8071.470215,115600
3,125.818,7.45982,30/07/2017,"Sun - Jul 30, 2017",20:50:00,MURDER,Magugpo North,Tagum City,Shooting,20:00 - 20:59,Sunday,50.546,8075.720215,8079.700195,8046.759766,8071.470215,8071.470215,115600
4,125.818,7.46016,30/07/2017,"Sun - Jul 30, 2017",20:50:00,PHYSICAL INJURIES,Magugpo North,Tagum City,Shooting,20:00 - 20:59,Sunday,50.546,8075.720215,8079.700195,8046.759766,8071.470215,8071.470215,115600


## SocioEconomic Data

In [20]:
socio_eco_data_temp = pd.read_csv('ddn_socio_eco_by_year.csv', encoding='utf-8')
socio_eco_data_temp.head()

,year,city,barangay,placetype,land area sq km,"Education,Baccalauate graduate","Education,College undergraduate","Education,Elementary","Education,High school","Education,No grade completed",...,"Population,male,55-59","Population,male,60-64","Population,male,65-69","Population,male,70-74","Population,male,75-79","Population,male,80 years and over","Poverty,poverty incidence by employed","Poverty,poverty incidence by household","Poverty,poverty incidence by population","Poverty,poverty incidence by unemployed"
0,2017,Panabo City,A. O. Floirendo,urban,6.60,443,387,610,"1,368",14,...,95,73,45,23,14,11,333,191,1090,19
1,2017,Panabo City,Datu Abdul Dadia,urban,7.88,529,462,728,"1,634",17,...,114,87,53,28,17,13,398,229,1303,22
2,2017,Panabo City,Buenavista,rural,1.09,73,64,101,226,2,...,16,12,7,4,2,2,55,32,180,3
3,2017,Panabo City,Cacao,rural,1.63,109,95,150,337,4,...,23,18,11,6,4,3,82,47,269,5
4,2017,Panabo City,Cagangohan,urban,18.74,"1,259","1,100","1,732","3,886",41,...,270,207,127,67,40,30,947,544,3098,53


In [21]:
import datetime
def get_year(date):
    date_str_format = "%d/%m/%Y"
    return datetime.datetime.strptime(str(date), date_str_format).year

In [22]:
crime_data_temp = crime_data.copy()
crime_data_temp['year'] = crime_data_temp.apply(lambda row: get_year(row['date']),axis=1)
crime_data_temp.head()

,lng,lat,date,customdate,time,customtime,crime,barangay,city,region,province,modus,crimetype,station,moduscode,date_com,datechar,year
0,125.808,7.44808,30/07/2017,"Sun - Jul 30, 2017",16:11:00,4:11:PM,Vehicular Traffic Accident,Magugpo Poblacion,Tagum City,110000000,112300000,Not Indicated,123,Tagum CPS,mc00,30/07/2017,42946,2017
1,125.803,7.44675,30/07/2017,"Sun - Jul 30, 2017",0:20:00,12:20:AM,MURDER,Magugpo Poblacion,Tagum City,110000000,112300000,Stabbing/Hacking,2,Tagum CPS,mc90,30/07/2017,42946,2017
2,125.803,7.44897,30/07/2017,"Sun - Jul 30, 2017",0:20:00,12:20:AM,PHYSICAL INJURIES,Magugpo Poblacion,Tagum City,110000000,112300000,Stabbing/Hacking,6,Tagum CPS,mc94,30/07/2017,42946,2017
3,125.818,7.45982,30/07/2017,"Sun - Jul 30, 2017",20:50:00,8:50:PM,MURDER,Magugpo North,Tagum City,110000000,112300000,Shooting,2,Tagum CPS,mc89,30/07/2017,42946,2017
4,125.818,7.46016,30/07/2017,"Sun - Jul 30, 2017",20:50:00,8:50:PM,PHYSICAL INJURIES,Magugpo North,Tagum City,110000000,112300000,Shooting,6,Tagum CPS,mc93,30/07/2017,42946,2017


In [23]:
socio_eco_data = socio_eco_data_temp.copy()
socio_eco_data = pd.merge(crime_data_temp, socio_eco_data, how='left', left_on=['year','barangay','city'], right_on=['year','barangay','city'])
socio_eco_data.drop(socio_eco_data.columns[[5,9,10,12,13,14,15,16,17]],axis=1,inplace=True)
socio_eco_data.to_csv('hello.csv')

## Population

In [24]:
pop_data = socio_eco_data.copy()
pop_data.drop(pop_data.columns[[11,12,13,14,15,16,17,18,19,20,21,
                                22,23,24,25,26,27,28,29,30,
                                31,32,33,34,35,36,37,38,39,
                                40,41,42,43,44,45,46,47,48,
                                49,50,51,52,53,54,55,56,57,
                                58,59,60,61,62,63,64,65,66,
                                67,68,69,70,71,72,73,74,75,
                                76,77,78,136,137,138,139]],axis=1,inplace=True)
pop_data.to_csv('generated/crime_pop.csv')

## Education

In [25]:
edu_data = socio_eco_data.copy()
edu_data = edu_data.iloc[:,:24]
edu_data.to_csv('generated/crime_educ.csv')

## Housing

In [26]:
house_data = socio_eco_data.copy()
house_data.drop(house_data.columns[[11,12,13,14,15,16,17,18,19,20,21,
                                    22,23,48,49,50,51,52,53,54,55,
                                    56,57,58,59,60,61,62,63,64,
                                    65,66,67,68,69,70,71,72,73,
                                    74,75,76,77,78,79,80,81,82,
                                    83,84,85,86,87,88,89,90,91,
                                    92,93,94,95,96,97,98,99,100,
                                    101,102,103,104,105,106,107,
                                    108,109,110,111,112,113,114,
                                    115,116,117,118,119,120,121,
                                    122,123,124,125,126,127,128,
                                    129,130,131,132,133,134,135,
                                    136,137,138,139]],axis=1,inplace=True)
house_data.to_csv('generated/crime_housing.csv')

## Marital Status

In [27]:
marital_data = socio_eco_data.copy()
marital_data.drop(marital_data.columns[[11,12,13,14,15,16,17,18,19,20,21,22,
                                      23,24,25,26,27,28,29,30,31,32,
                                      33,34,35,36,37,38,39,40,41,42,
                                      43,44,45,46,47,48,49,50,51,52,
                                      53,54,55,56,57,58,59,60,79,
                                      80,81,82,83,84,85,86,87,88,89,
                                      90,91,92,93,94,95,96,97,98,99,
                                      100,101,102,103,104,105,106,107,
                                      108,109,110,111,112,113,114,115,
                                      116,117,118,119,120,121,122,123,
                                      124,125,126,127,128,129,130,131,
                                      132,133,134,135,136,137,138,139]],axis=1,inplace=True)
marital_data.to_csv('generated/crime_marital.csv')

## Labor

In [28]:
labor_data = socio_eco_data.copy()
labor_data.drop(labor_data.columns[[11,12,13,14,15,16,17,18,19,20,21,22,23,
                                    24,25,26,27,28,29,30,31,32,33,34,35,
                                    36,37,38,39,40,41,42,43,44,45,46,47,
                                    61,62,63,64,65,66,67,68,69,70,71,72,
                                    73,74,75,76,77,78,79,80,81,82,83,84,
                                    85,86,87,88,89,90,91,92,93,94,95,96,
                                    97,98,99,100,101,102,103,104,105,106,
                                    107,108,109,110,111,112,113,114,115,116,
                                    117,118,119,120,121,122,123,124,125,126,
                                    127,128,129,130,131,132,133,134,135,136,137,138,139]],axis=1,inplace=True)
labor_data.to_csv('generated/crime_labor.csv')

## Poverty

In [29]:
poverty_data = socio_eco_data.copy()
poverty_data.drop(poverty_data.columns[[11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,
                                        26,27,28,29,30,31,32,33,34,35,36,37,38,39,
                                        40,41,42,43,44,45,46,47,48,49,50,51,52,53,
                                        54,55,56,57,58,59,60,61,62,63,64,65,66,67,
                                        68,69,70,71,72,73,74,75,76,77,78,79,80,81,
                                        82,83,84,85,86,87,88,89,90,91,92,93,94,95,
                                        96,97,98,99,100,101,102,103,104,105,106,107,
                                        108,109,110,111,112,113,114,115,116,117,118,
                                        119,120,121,122,123,124,125,126,127,128,129,
                                        130,131,132,133,134,135]],axis=1,inplace=True)
poverty_data.to_csv('generated/crime_poverty.csv')